In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon
import folium
from folium import GeoJson
from folium import Choropleth
from folium.plugins import HeatMap
import numpy as np
from haversine import haversine, Unit
import matplotlib.pyplot as plt
import json
from datetime import timedelta
import requests
import os
from dotenv import load_dotenv
from geopy.distance import distance

In [ ]:
bus_routes = pd.read_csv("data/bus_routes_full.csv")
bus_stops = pd.read_csv("data/bus_stops_full.csv")
passenger_volume_OD = pd.read_csv("data/passenger_volume_OD.csv")
passenger_volume = pd.read_csv("data/passenger_volume.csv")
planning_areas = gpd.read_file("data/MasterPlan2019PlanningAreaBoundaryNoSea.geojson")
mrt_stations = pd.read_csv("data/mrt_stations.csv")


In [ ]:
bus_routes_combined = pd.merge(bus_routes, bus_stops, on='BusStopCode', how='left')
bus_routes_combined.head()

In [ ]:
bus_routes_combined['geometry'] = bus_routes_combined.apply(lambda x: Point((x.Longitude, x.Latitude)), axis=1)

In [ ]:
bus_routes_combined = gpd.GeoDataFrame(bus_routes_combined, geometry='geometry')

bus_routes_combined = bus_routes_combined.set_crs(epsg=4326)

print(bus_routes_combined.head())

  ServiceNo Operator  Direction  StopSequence  BusStopCode  Distance  \
0        10     SBST          1             1        75009       0.0   
1        10     SBST          1             2        76059       0.6   
2        10     SBST          1             3        76069       1.1   
3        10     SBST          1             4        96289       2.3   
4        10     SBST          1             5        96109       2.7   

  WD_FirstBus WD_LastBus SAT_FirstBus SAT_LastBus SUN_FirstBus SUN_LastBus  \
0        0500       2300         0500        2300         0500        2300   
1        0502       2302         0502        2302         0502        2302   
2        0504       2304         0504        2304         0503        2304   
3        0508       2308         0508        2309         0507        2308   
4        0509       2310         0509        2311         0508        2309   

          RoadName           Description  Latitude   Longitude  \
0  Tampines Ctrl 1          Tamp

In [ ]:
bus_routes_combined.to_csv("data/bus_routes_combined.csv", index=False)

In [ ]:
routes = (
    bus_routes_combined.sort_values(by=['ServiceNo', 'Direction', 'StopSequence'])  # Ensure stops are in the correct order
    .groupby(['ServiceNo', 'Direction'])['geometry']
    .apply(lambda x: LineString(x.tolist()))  # Create LineString from points
    .reset_index()
)

bus_routes_ls = gpd.GeoDataFrame(routes, geometry='geometry', crs="EPSG:4326")

In [ ]:
print(bus_routes_ls.head())
bus_routes_ls.to_file("data/line_string.geojson", driver='GeoJSON')

  ServiceNo  Direction                                           geometry
0        10          1  LINESTRING (103.94339 1.35408, 103.94165 1.352...
1        10          2  LINESTRING (103.76988 1.29425, 103.76908 1.292...
2       100          1  LINESTRING (103.87169 1.35047, 103.87205 1.346...
3       100          2  LINESTRING (103.78932 1.31107, 103.78969 1.309...
4      100A          1  LINESTRING (103.87169 1.35047, 103.87205 1.346...


In [ ]:
# Function to split and duplicate entries
def duplicate_entries(df):
    # Create an empty list to store new rows
    new_rows = []

    for _, row in df.iterrows():
        # Check if the STN_NO contains a '/'
        if '/' in row['STN_NO']:
            # Split the STN_NO
            lines = row['STN_NO'].split('/')
            # Create a new row for each line
            for line in lines:
                new_row = row.copy()
                new_row['STN_NO'] = line.strip()  # Update the STN_NO to the new line
                new_rows.append(new_row)
        else:
            new_rows.append(row)

    # Create a new DataFrame from the new rows
    return pd.DataFrame(new_rows)

# Apply the function to duplicate entries
mrt_stations_expanded = duplicate_entries(mrt_stations)

mrt_stations_expanded['NO'] = mrt_stations_expanded['NO'].astype(int)

mrt_stations_clean = (mrt_stations_expanded.sort_values(['LINE', 'NO']).reset_index(drop=True))

print(mrt_stations_clean.head())

                      STN_NAME STN_NO  Latitude   Longitude LINE  NO
0     DHOBY GHAUT MRT STATION     CC1  1.298912  103.846293   CC   1
1      BRAS BASAH MRT STATION     CC2  1.296862  103.850667   CC   2
2       ESPLANADE MRT STATION     CC3  1.293658  103.855081   CC   3
3       PROMENADE MRT STATION     CC4  1.293998  103.860350   CC   4
4  NICOLL HIGHWAY MRT STATION     CC5  1.299767  103.863637   CC   5


In [ ]:
# Create GeoDataFrame for MRT Stations
mrt_gdf = gpd.GeoDataFrame(
   mrt_stations_clean,
    geometry=gpd.points_from_xy(mrt_stations_clean.Longitude, mrt_stations_clean.Latitude),
    crs="EPSG:4326"  # WGS84 Latitude/Longitude
)

                         STN_NAME STN_NO  Latitude   Longitude LINE  NO  \
171  WOODLANDS NORTH MRT STATION     TE1  1.448292  103.785693   TE   1   
172        WOODLANDS MRT STATION     TE2  1.436058  103.787939   TE   2   
173  WOODLANDS SOUTH MRT STATION     TE3  1.427396  103.793264   TE   3   
174       SPRINGLEAF MRT STATION     TE4  1.397581  103.817857   TE   4   
175           LENTOR MRT STATION     TE5  1.385507  103.835744   TE   5   

                      geometry  
171  POINT (103.78569 1.44829)  
172  POINT (103.78794 1.43606)  
173   POINT (103.79326 1.4274)  
174  POINT (103.81786 1.39758)  
175  POINT (103.83574 1.38551)  


DTL

In [ ]:
DT_stations = mrt_stations[mrt_stations['STN_NO'].str.contains(r'\bDT', regex=True)]
DT_stations = DT_stations.assign(DT_Code=DT_stations['STN_NO'].str.extract(r'(DT\d+)'))
DT_stations = DT_stations.assign(DT_Number = DT_stations['DT_Code'].str.extract(r'(\d+)').astype(int))
DT_sorted = DT_stations.sort_values(by='DT_Number').reset_index(drop=True)
DT_sorted = DT_sorted.drop(columns=['DT_Code','DT_Number'])
DT_sorted['geometry'] = DT_sorted.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
DT_sorted = gpd.GeoDataFrame(DT_sorted, geometry='geometry')

route_line = LineString(DT_sorted['geometry'].tolist())
DT_ls = gpd.GeoDataFrame({'Line': ['DTL'], 'geometry': [route_line]}, crs="EPSG:4326")

DT_ls = DT_ls.to_crs(epsg=32648)
bus_routes_ls = bus_routes_ls.to_crs(epsg=32648)

DT_buffer_400= DT_ls.buffer(400).union_all()

In [ ]:
def calculate_overlap(route):
    intersection = route.intersection(DT_buffer_400)
    overlap_length = intersection.length
    route_length = route.length
    overlap_percentage = (overlap_length/route_length)*100 if route_length > 0 else 0
    return pd.Series({'Overlap Length': overlap_length, 'Overlap Percentage': overlap_percentage})

In [ ]:
def calculate_overlap(route):
    intersection = route.intersection(DT_buffer_400)
    overlap_length = intersection.length
    route_length = route.length
    overlap_percentage = (overlap_length/route_length)*100 if route_length > 0 else 0
    return pd.Series({'Overlap Length': overlap_length, 'Overlap Percentage': overlap_percentage})

In [ ]:
service_overlap = buffer_overlap_400.groupby(['ServiceNo', 'Direction'])[['Overlap Length', 'Overlap Percentage']].sum().reset_index()
print(service_overlap)

In [ ]:
service_overlap_sorted = service_overlap.sort_values(by='Overlap Percentage', ascending=False)
service_overlap_sorted.head(10)

In [ ]:
top5_overlap = service_overlap_sorted.head(5)['ServiceNo'].tolist()
top5_bus_routes = routes[routes['ServiceNo'].isin(top5_overlap)]

In [ ]:
sgmap = folium.Map(location=[1.3521, 103.8198], zoom_start=12)

coords = [(lat, lon) for lon,lat in route_line.coords]

folium.PolyLine(
    locations = coords,
    color='blue',
    weight=5,
    opacity=0.8,
    tooltip="Downtown Line"
).add_to(sgmap)

for _,row in DT_sorted.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['STN_NAME'],
        icon=folium.Icon(color='blue', icon='train', prefix='fa')
    ).add_to(sgmap)

map_buffer = route_line.buffer(0.0036)
gdf_buffer = gpd.GeoDataFrame(geometry=[map_buffer], crs=DT_sorted.crs)
buffer_geojson = gdf_buffer.to_json()

folium.GeoJson(
    buffer_geojson,
    style_function=lambda x: {
        'fillColor': 'blue',
        'color': 'blue',
        'weight': 1,
        'fillOpacity': 0.5
    },
).add_to(sgmap)

colors = {
    '67':'red',
    '23':'green',
    '170':'orange'
}

for _, row in top5_bus_routes.iterrows():
    coords = [(lat, lon) for lon,lat in row['geometry'].coords]
    folium.PolyLine(
        locations = coords,
        color=colors.get(row['ServiceNo'], 'black'),
        weight=5,
        opacity=0.8,
        tooltip=f"{row['ServiceNo']} - Direction {row['Direction']}"
    ).add_to(sgmap)

    start_point = coords[0]
    end_point = coords[-1]

    folium.Marker(location=start_point, icon=folium.Icon(color=colors.get(row['ServiceNo'], 'black'))).add_to(sgmap)
    folium.Marker(location=end_point, icon=folium.Icon(color=colors.get(row['ServiceNo'], 'black'))).add_to(sgmap)

sgmap

Identified Bus 23

In [ ]:
bus_routes_combined[bus_routes_combined['ServiceNo'] == '23']

In [ ]:
sgmap2 = folium.Map(location=[1.3521, 103.8198], zoom_start=12)

coords = [(lat, lon) for lon,lat in route_line.coords]

folium.PolyLine(
    locations = coords,
    color='blue',
    weight=5,
    opacity=0.8,
    tooltip="Downtown Line"
).add_to(sgmap2)

for _,row in DT_sorted.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['STN_NAME'],
        icon=folium.Icon(color='blue', icon='train', prefix='fa')
    ).add_to(sgmap2)

# Add Bus 23 stops
# these are the first half of the stops
for _, row in bus_routes_combined[bus_routes_combined['ServiceNo'] == '23'][:24].iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['BusStopCode'],
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(sgmap2)

# these are the 2nd half of the stops
for _, row in bus_routes_combined[bus_routes_combined['ServiceNo'] == '23'][24:].iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['BusStopCode'],
        icon=folium.Icon(color='green', icon='info-sign')
    ).add_to(sgmap2)

map_buffer = route_line.buffer(0.0036)
gdf_buffer = gpd.GeoDataFrame(geometry=[map_buffer], crs=DT_sorted.crs)
buffer_geojson = gdf_buffer.to_json()

folium.GeoJson(
    buffer_geojson,
    style_function=lambda x: {
        'fillColor': 'blue',
        'color': 'blue',
        'weight': 1,
        'fillOpacity': 0.5
    },
).add_to(sgmap2)

sgmap2

check if there are any alternative bus routes that go from inside the 23 buffer to outside, following the same route as 23 for those stops that are outside the buffer

In [ ]:
# Step 1: Filter for Bus 23 stops
bus_23_stops = bus_routes_combined[bus_routes_combined['ServiceNo'] == '23'].copy()

# Check if the bus stops are within the buffer
bus_23_stops['within_dt_buffer'] = bus_23_stops.geometry.apply(
    lambda stop: DT_buffer_400.intersects(stop)
)

# Step 2: Assign groups for outside bus stops
bus_23_stops['outside_group'] = (bus_23_stops['within_dt_buffer'] != bus_23_stops['within_dt_buffer'].shift()).cumsum()

# Step 3: Get unique outside groups
unique_outside_groups_23 = bus_23_stops['outside_group'].dropna().unique()

# Step 4: Initialize a dictionary to hold the results for Bus 23
other_bus_routes_23 = {}

# Step 5: Loop through each outside group to find other bus services
for group in unique_outside_groups_23:
    # Get bus stop codes for the current outside group
    outside_stop_codes_23 = bus_23_stops[bus_23_stops['outside_group'] == group]['BusStopCode'].unique()

    # Find the last bus stop that was within the buffer for this group
    within_buffer_stops = bus_23_stops[(bus_23_stops['outside_group'] == group) & (bus_23_stops['within_dt_buffer'])]

    # Check if there are any stops within the buffer
    if not within_buffer_stops.empty:
        last_within_buffer_23 = within_buffer_stops.iloc[-1]
        last_stop_code_23 = last_within_buffer_23['BusStopCode']

        # Step 6: Find other bus routes that service the last stop and the outside stops
        other_routes_23 = bus_routes_combined[
            bus_routes_combined['BusStopCode'].isin([last_stop_code_23] + list(outside_stop_codes_23))
        ]['ServiceNo'].unique()

        # Store the results
        other_bus_routes_23[group] = other_routes_23
    else:
        other_bus_routes_23[group] = []  # If there's no valid last stop within the buffer

# Step 7: Print the results for Bus 23
for group, routes in other_bus_routes_23.items():
    print(f"Outside Group {group} for Bus 23: Other Bus Routes: {routes}")


Bus 67

In [ ]:
route_67 = routes[routes['ServiceNo'] == '67']
bus_67 = bus_routes_combined[bus_routes_combined['ServiceNo'] == '67']
bus_67_stops = bus_67[['ServiceNo', 'Direction', 'StopSequence', 'BusStopCode', 'Description', 'Latitude', 'Longitude', 'geometry']]

# To find the number of stops within and outside the buffer
def stops_within_buffer_by_direction(bus_service, buffer):
    results = {}

    for direction, group in bus_service.groupby('Direction'):
        group['Within Buffer'] = group['geometry'].apply(lambda x: x.within(buffer))

        within=group[group['Within Buffer']]
        outside=group[~group['Within Buffer']]

        results[direction] = {
            'within': len(within),
            'outside': len(outside),
            'within_stops': within,
            'outside_stops': outside
        }

    return results

results_67 = stops_within_buffer_by_direction(bus_67, DT_buffer)
for direction, result in results_67.items():
    print(f"Bus 67 - Direction {direction}:")
    print(f"Stops within buffer: {result['within']}")
    print(f"Stops outside buffer: {result['outside']}")

within_d1 = results_67[1]['within_stops']
outside_d1 = results_67[1]['outside_stops']

within_d2 = results_67[2]['within_stops']
outside_d2 = results_67[2]['outside_stops']


In [ ]:
#To add Bus 67's route to the folium map

colors = {
    1: 'purple',
    2: 'darkgreen'
}

for _,row in route_67.iterrows():
    coords = [(lat,lon) for lon,lat in row['geometry'].coords]
    direction = row['Direction']
    color = colors.get(direction, 'black')

    folium.PolyLine(
        locations = coords,
        color = color,
        weight = 4,
        opacity = 0.8,
        tooltip = f"Service 67 - Direction {direction}"
    ).add_to(sgmap2)

for _, stop in bus_67_stops.iterrows():
    stop_coords = [stop.geometry.y, stop.geometry.x]
    direction = stop['Direction']
    color = colors.get(direction, 'black')

    folium.Marker(
        location = stop_coords,
        popup = f"Stop: {stop['Description']}<br>Stop Code: {stop['BusStopCode']}",
        tooltip = f"Bus Stop: {stop['Description']} - Direction {direction}",
        icon = folium.Icon(color=color, icon='bus', prefix='fa')
    ).add_to(sgmap2)

Route information from OneMap API

In [ ]:
load_dotenv()
onemap_api_key = os.getenv('onemap_api_key')

In [ ]:
import requests
from datetime import timedelta
# Define constant parameters
DATE = '10-21-2024'
TIME = '08:30:00'
MODE = 'TRANSIT'
NUM_ITINERARIES = '5'
MAX_WALK = '400'

# Assume that the API token is defined
API_TOKEN = onemap_api_key

# Function to get bus route data
def get_route(start_coords, end_coords):
    url = "https://www.onemap.gov.sg/api/public/routingsvc/route"
    headers = {
        "Authorization": API_TOKEN
    }
    params = {
        "start": f"{start_coords[0]},{start_coords[1]}",
        "end": f"{end_coords[0]},{end_coords[1]}",
        "routeType": "pt",
        "date": DATE,
        "time": TIME,
        "mode": MODE,
        "numItineraries": NUM_ITINERARIES,
        "maxWalkDistance": MAX_WALK,
    }
    try:
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            return response.json()  # Return the entire JSON response
        else:
            print(f"Error: Received status code {response.status_code}")
            print(f"Response: {response.text}")  # Log the response for debugging
    except requests.exceptions.RequestException as e:
        print(f"Error making request to OneMap API: {e}")
    return None

def format_duration(seconds):
    """Convert seconds to a formatted string of hours, minutes, and seconds."""
    return str(timedelta(seconds=seconds))

def print_itineraries(data):
    """
    Extracts and prints itinerary details from OneMap API response data.

    Parameters:
    - data: dict, the JSON response from the OneMap API, converted to a dictionary.
    """
    # Get the itineraries from the response
    itineraries = data.get('plan', {}).get('itineraries', [])

    if not itineraries:
        print("No itineraries found.")
        return

    for idx, itinerary in enumerate(itineraries):
        # Itinerary-level details
        total_duration = itinerary.get('duration', 0)
        transit_time = itinerary.get('transitTime', 0)
        walk_time = itinerary.get('walkTime', 0)
        fare = itinerary.get('fare', 'N/A')

        # Format duration and times
        formatted_duration = format_duration(total_duration)
        formatted_transit_time = format_duration(transit_time)
        formatted_walk_time = format_duration(walk_time)
        formatted_fare = f"${fare:.2f}" if isinstance(fare, (int, float)) else fare

        # Print summary of the itinerary
        print(f"\n--- Itinerary {idx + 1} ---")
        print(f"Total Duration: {formatted_duration}")
        print(f"Transit Time: {formatted_transit_time}")
        print(f"Walk Time: {formatted_walk_time}")
        print(f"Fare: {formatted_fare}")

        # Iterate over the legs in the itinerary
        for leg_idx, leg in enumerate(itinerary.get('legs', [])):
            mode = leg.get('mode', 'N/A')
            route = leg.get('route', 'N/A')
            from_stop = leg.get('from', {}).get('name', 'Unknown')
            to_stop = leg.get('to', {}).get('name', 'Unknown')
            distance = leg.get('distance', 0)
            start_time = leg.get('startTime', 'Unknown')
            end_time = leg.get('endTime', 'Unknown')
            leggeometry = leg.get('legGeometry', {}).get('points', [])
            num_stops = len(leg.get('intermediateStops', []))  # Count intermediate stops

            # Print details for each leg
            print(f"\nLeg {leg_idx + 1}:")
            print(f"  Mode: {mode}")
            print(f"  Route: {route if route else 'N/A'}")
            print(f"  From: {from_stop}")
            print(f"  To: {to_stop}")
            print(f"  Distance: {distance:.2f} meters")
            print(f"  Start Time: {start_time}")
            print(f"  End Time: {end_time}")
            if num_stops > 0:
                print(f"  Number of Intermediate Stops: {num_stops}")
            else:
                print(f"  No Intermediate Stops")
            if leggeometry:
                print(f"  Leg Geometry: {leggeometry} ")

    print("\n--- End of Itineraries ---")

In [ ]:
def get_coordinates(bus_stop_code):
    # Look for the row in bus_stops_36B that matches the bus_stop_code
    stop = bus_stops[bus_stops['BusStopCode'] == bus_stop_code]

    # If a matching row is found, return the coordinates (latitude, longitude)
    if not stop.empty:
        lat = stop.iloc[0]['Latitude']
        lon = stop.iloc[0]['Longitude']
        return (lat, lon)
    else:
        return None  # Return None if the bus stop code is not found

In [ ]:
# Define start and end coordinates
#start_coords =  get_coordinates(75229)  # Example starting point
start_coords = (1.3463354,103.9339796)
#end_coords = get_coordinates(60099)
end_coords = (1.3213153,103.8732946)  # Example ending point

print(start_coords)
print(end_coords)

data = get_route(start_coords, end_coords)
print(data)
print_itineraries(data)  # Call the function to print itineraries

passenger demand for bus 23

In [ ]:
passenger_volume_OD_sept = pd.read_csv("data/passenger_volume_OD_by_bus_stops.csv")
passenger_volume_OD_aug = pd.read_csv("data/origin_destination_bus_202408.csv")
passenger_volume_OD_july = pd.read_csv("data/origin_destination_bus_202407.csv")

# Combine the filtered datasets using pd.concat
passenger_volume_OD = pd.concat([passenger_volume_OD_sept,
                                       passenger_volume_OD_aug,
                                       passenger_volume_OD_july])

# Filter for time period 8.0 (8 AM) and weekday
filtered_passenger_volume_OD = passenger_volume_OD[
    (passenger_volume_OD['TIME_PER_HOUR'].isin([8,9])) &
    (passenger_volume_OD["DAY_TYPE"] == "WEEKDAY") &
    (passenger_volume_OD["ORIGIN_PT_CODE"] == 93019) &
    (passenger_volume_OD_sept["DESTINATION_PT_CODE"].isin([2159, 4111, 9022]))
]

summary = filtered_passenger_volume_OD.groupby(
    ['ORIGIN_PT_CODE', 'DESTINATION_PT_CODE', 'TIME_PER_HOUR', 'YEAR_MONTH']
).agg({'TOTAL_TRIPS': 'sum'}).reset_index()

# Display the summarized data
# print(summary)

# Pivot the DataFrame to have origin-destination pairs as rows and time-period columns
summary_pivot = summary.pivot_table(
    index=['ORIGIN_PT_CODE', 'DESTINATION_PT_CODE'],  # Rows: origin-destination pairs
    columns=['YEAR_MONTH', 'TIME_PER_HOUR'],  # Columns: month and time period
    values='TOTAL_TRIPS',  # Values: total trips
    aggfunc='sum',  # Aggregation function
).fillna(0)  # Fill NaN values with 0 (for combinations with no trips)

# Rename the columns to have more descriptive labels (e.g., total_trips_july_8)
summary_pivot.columns = [
    f"total_trips_{year_month}_{int(time_hour)}"
    for year_month, time_hour in summary_pivot.columns
]

# Reset index to make it a regular DataFrame
summary_pivot.reset_index(inplace=True)

# Display the pivoted DataFrame
print(summary_pivot)

average tap in tap out for 23

In [ ]:
def get_passenger_demand(bus_routes_combined, passenger_volume, year_month, service_no):
    # Filter the passenger_volume DataFrame
    passenger_volume_am = passenger_volume[
        (passenger_volume['TIME_PER_HOUR'].isin([8, 9])) &
        (passenger_volume["DAY_TYPE"] == "WEEKDAY") &
        (passenger_volume['YEAR_MONTH'] == year_month)
    ]

    # Group by PT_CODE and sum the relevant passenger volumes
    passenger_volume_am = passenger_volume_am.groupby('PT_CODE').agg({
        'TOTAL_TAP_IN_VOLUME': 'sum',
        'TOTAL_TAP_OUT_VOLUME': 'sum'
    }).reset_index()

    # Count the number of services for each bus stop
    service_counts = bus_routes_combined.groupby('BusStopCode')['ServiceNo'].nunique().reset_index()
    service_counts.columns = ['BusStopCode', 'ServiceCount']

    avg_tap_volume = pd.merge(service_counts, passenger_volume_am, left_on="BusStopCode", right_on='PT_CODE', how='left')

    # Clean up duplicate PT_CODE columns (from both merges)
    avg_tap_volume.drop(columns=['PT_CODE'], inplace=True)

    avg_tap_volume['avg_tap_in'] = avg_tap_volume['TOTAL_TAP_IN_VOLUME'] / avg_tap_volume['ServiceCount']
    avg_tap_volume['avg_tap_out'] = avg_tap_volume['TOTAL_TAP_OUT_VOLUME'] / avg_tap_volume['ServiceCount']

    # Fill NaN values with 0 for average calculations
    avg_tap_volume.fillna(0, inplace=True)

    avg_tap_volume['avg_tap_in'] = avg_tap_volume['avg_tap_in'].round().astype(int)
    avg_tap_volume['avg_tap_out'] = avg_tap_volume['avg_tap_out'].round().astype(int)

    bus_stops_passenger_demand = bus_routes_combined.merge(
        avg_tap_volume, on='BusStopCode', how='left'
    )

    # Filter for the specified service number and direction (1)
    passenger_demand = bus_stops_passenger_demand[
        (bus_stops_passenger_demand['ServiceNo'] == service_no) &
        (bus_stops_passenger_demand['Direction'] == 1)
    ]

    # Ensure the average tap-in volumes are rounded to whole numbers
    passenger_demand['avg_tap_in'] = passenger_demand['avg_tap_in'].round()
    passenger_demand['avg_tap_out'] = passenger_demand['avg_tap_out'].round()

    # Set the last observation of avg_tap_in to 0
    passenger_demand['avg_tap_in'].iloc[-1] = 0
    passenger_demand['avg_tap_out'].iloc[0] = 0

    # Sort the DataFrame by StopSequence for ordered plotting
    passenger_demand = passenger_demand.sort_values(by='StopSequence')

    # Convert BusStopCode to a categorical variable ordered by StopSequence
    passenger_demand['BusStopCode'] = pd.Categorical(passenger_demand['BusStopCode'],
                                                       categories=passenger_demand['BusStopCode'].unique(),
                                                       ordered=True)

    return passenger_demand

In [ ]:
import matplotlib.pyplot as plt

# Get passenger demand data for the three months
service_no = "23"
passenger_demand_23_jul = get_passenger_demand(bus_routes_combined, passenger_volume, "2024-07", service_no)
passenger_demand_23_aug = get_passenger_demand(bus_routes_combined, passenger_volume, "2024-08", service_no)
passenger_demand_23_sept = get_passenger_demand(bus_routes_combined, passenger_volume, "2024-09", service_no)

# Create side-by-side subplots
fig, axs = plt.subplots(1, 2, figsize=(16, 6))

# Plot average tap-in volumes
axs[0].plot(range(len(passenger_demand_23_jul)), passenger_demand_23_jul['avg_tap_in'], marker='o', color='blue', linestyle='-', label='July')
axs[0].plot(range(len(passenger_demand_23_aug)), passenger_demand_23_aug['avg_tap_in'], marker='o', color='orange', linestyle='-', label='August')
axs[0].plot(range(len(passenger_demand_23_sept)), passenger_demand_23_sept['avg_tap_in'], marker='o', color='green', linestyle='-', label='September')

# Add labels and title for tap-in plot
axs[0].set_xlabel('Bus Stop Code')
axs[0].set_ylabel('Average Tap-In Volume')
axs[0].set_title('Average Tap-In Volume for Bus Service 23 by Stop Sequence')
axs[0].set_xticks(range(len(passenger_demand_23_jul)))
axs[0].set_xticklabels(passenger_demand_23_jul['BusStopCode'],  rotation=90, ha='right')
axs[0].grid(axis='y')
axs[0].legend()

# Plot average tap-out volumes
axs[1].plot(range(len(passenger_demand_23_jul)), passenger_demand_23_jul['avg_tap_out'], marker='o', color='blue', linestyle='-', label='July')
axs[1].plot(range(len(passenger_demand_23_aug)), passenger_demand_23_aug['avg_tap_out'], marker='o', color='orange', linestyle='-', label='August')
axs[1].plot(range(len(passenger_demand_23_sept)), passenger_demand_23_sept['avg_tap_out'], marker='o', color='green', linestyle='-', label='September')

# Add labels and title for tap-out plot
axs[1].set_xlabel('Bus Stop Code')
axs[1].set_ylabel('Average Tap-Out Volume')
axs[1].set_title('Average Tap-Out Volume for Bus Service 23 by Stop Sequence')
axs[1].set_xticks(range(len(passenger_demand_23_jul)))
axs[1].set_xticklabels(passenger_demand_23_jul['BusStopCode'],  rotation=90, ha='right')
axs[1].grid(axis='y')
axs[1].legend()

# Show the plots
plt.tight_layout()  # Adjust layout to prevent clipping of tick-labels
plt.show()

Passenger Demand for Bus 67

In [ ]:
passenger_volume_OD_sept = pd.read_csv("data/passenger_volume_OD_by_bus_stops.csv")
passenger_volume_OD_aug = pd.read_csv("data/origin_destination_bus_202408.csv")
passenger_volume_OD_july = pd.read_csv("data/origin_destination_bus_202407.csv")

# Combine the filtered datasets using pd.concat
passenger_volume_OD = pd.concat([passenger_volume_OD_sept,
                                       passenger_volume_OD_aug,
                                       passenger_volume_OD_july])

# Filter for time period 8.0 (8 AM) and weekday
filtered_passenger_volume_OD = passenger_volume_OD[
    (passenger_volume_OD['TIME_PER_HOUR'].isin([8,9])) &
    (passenger_volume_OD["DAY_TYPE"] == "WEEKDAY") &
    (passenger_volume_OD["ORIGIN_PT_CODE"] == 44009) &
    (passenger_volume_OD_sept["DESTINATION_PT_CODE"].isin([2159, 4111, 9022]))
]

summary = filtered_passenger_volume_OD.groupby(
    ['ORIGIN_PT_CODE', 'DESTINATION_PT_CODE', 'TIME_PER_HOUR', 'YEAR_MONTH']
).agg({'TOTAL_TRIPS': 'sum'}).reset_index()

# Display the summarized data
# print(summary)

# Pivot the DataFrame to have origin-destination pairs as rows and time-period columns
summary_pivot = summary.pivot_table(
    index=['ORIGIN_PT_CODE', 'DESTINATION_PT_CODE'],  # Rows: origin-destination pairs
    columns=['YEAR_MONTH', 'TIME_PER_HOUR'],  # Columns: month and time period
    values='TOTAL_TRIPS',  # Values: total trips
    aggfunc='sum',  # Aggregation function
).fillna(0)  # Fill NaN values with 0 (for combinations with no trips)

# Rename the columns to have more descriptive labels (e.g., total_trips_july_8)
summary_pivot.columns = [
    f"total_trips_{year_month}_{int(time_hour)}"
    for year_month, time_hour in summary_pivot.columns
]

# Reset index to make it a regular DataFrame
summary_pivot.reset_index(inplace=True)

# Display the pivoted DataFrame
print(summary_pivot)

In [ ]:
# Get passenger demand data for the three months
service_no = "67"
passenger_demand_67_jul = get_passenger_demand(bus_routes_combined, passenger_volume, "2024-07", service_no)
passenger_demand_67_aug = get_passenger_demand(bus_routes_combined, passenger_volume, "2024-08", service_no)
passenger_demand_67_sept = get_passenger_demand(bus_routes_combined, passenger_volume, "2024-09", service_no)

# Create side-by-side subplots
fig, axs = plt.subplots(1, 2, figsize=(16, 6))

# Plot average tap-in volumes
axs[0].plot(range(len(passenger_demand_67_jul)), passenger_demand_67_jul['avg_tap_in'], marker='o', color='blue', linestyle='-', label='July')
axs[0].plot(range(len(passenger_demand_67_aug)), passenger_demand_67_aug['avg_tap_in'], marker='o', color='orange', linestyle='-', label='August')
axs[0].plot(range(len(passenger_demand_67_sept)), passenger_demand_67_sept['avg_tap_in'], marker='o', color='green', linestyle='-', label='September')

# Add labels and title for tap-in plot
axs[0].set_xlabel('Bus Stop Code')
axs[0].set_ylabel('Average Tap-In Volume')
axs[0].set_title('Average Tap-In Volume for Bus Service 67 by Stop Sequence')
axs[0].set_xticks(range(len(passenger_demand_67_jul)))
axs[0].set_xticklabels(passenger_demand_67_jul['BusStopCode'],  rotation=90, ha='right')
axs[0].grid(axis='y')
axs[0].legend()

# Plot average tap-out volumes
axs[1].plot(range(len(passenger_demand_67_jul)), passenger_demand_67_jul['avg_tap_out'], marker='o', color='blue', linestyle='-', label='July')
axs[1].plot(range(len(passenger_demand_67_aug)), passenger_demand_67_aug['avg_tap_out'], marker='o', color='orange', linestyle='-', label='August')
axs[1].plot(range(len(passenger_demand_67_sept)), passenger_demand_67_sept['avg_tap_out'], marker='o', color='green', linestyle='-', label='September')

# Add labels and title for tap-out plot
axs[1].set_xlabel('Bus Stop Code')
axs[1].set_ylabel('Average Tap-Out Volume')
axs[1].set_title('Average Tap-Out Volume for Bus Service 67 by Stop Sequence')
axs[1].set_xticks(range(len(passenger_demand_67_jul)))
axs[1].set_xticklabels(passenger_demand_67_jul['BusStopCode'],  rotation=90, ha='right')
axs[1].grid(axis='y')
axs[1].legend()

# Show the plots
plt.tight_lay

In [ ]:
teline_gdf = mrt_gdf[mrt_gdf["LINE"] == "TE"]

# Display the first few rows to confirm the result
print(teline_gdf.head())

teline_gdf.to_file("data/teline_gdf.geojson", driver='GeoJSON')

route_line = LineString(teline_gdf['geometry'].tolist())
TE_ls = gpd.GeoDataFrame({'Line': ['MRT'], 'geometry': [route_line]}, crs="EPSG:4326")

TE_ls.to_file("data/TE_ls.geojson", driver='GeoJSON')

TE_ls = TE_ls.to_crs(epsg=32648)
bus_routes_ls = bus_routes_ls.to_crs(epsg=32648)

TE_buffer_400= TE_ls.buffer(400).union_all()
print(TE_buffer_400)

# Plotting the polygon
fig, ax = plt.subplots()
x, y = TE_buffer_400.exterior.xy  # Extract x and y coordinates
ax.plot(x, y, color='blue', linewidth=2)  # Plot the polygon outline
ax.fill(x, y, color='skyblue', alpha=0.5)  # Optional: Fill the polygon with color

# Set plot title and labels
ax.set_title('Polygon Plot')
ax.set_xlabel('X')
ax.set_ylabel('Y')

# Display the plot
plt.show()

Calculating the overlap between bus routes and TEL buffer

In [ ]:
def calculate_overlap(route, buffer):
    intersection = route.intersection(buffer)
    overlap_length = intersection.length
    route_length = route.length
    overlap_percentage = (overlap_length / route_length) * 100 if route_length > 0 else 0
    return pd.Series({'Overlap Length': overlap_length, 'Overlap Percentage': overlap_percentage})

Obtaining the top 10 bus routes with the most overlap with TEL

In [ ]:

TEL_buffer_overlap_400 = bus_routes_ls.copy()

TEL_buffer_overlap_400[['Overlap Length', 'Overlap Percentage']] = TEL_buffer_overlap_400['geometry'].apply(calculate_overlap, buffer=TE_buffer_400)

# print(TEL_buffer_overlap_400.head())

TEL_service_overlap = TEL_buffer_overlap_400.groupby(['ServiceNo', 'Direction'])[['Overlap Length', 'Overlap Percentage']].sum().reset_index()
# print(TEL_service_overlap)

TEL_service_overlap_sorted = TEL_service_overlap.sort_values(by='Overlap Percentage', ascending=False)
TEL_service_overlap_sorted.head(10)

TEL_top10_overlap = TEL_service_overlap_sorted.head(10)['ServiceNo'].tolist()
TEL_top10_bus_routes = routes[routes['ServiceNo'].isin(TEL_top10_overlap)]
print(TEL_top10_bus_routes)

#TEL_top10_bus_routes.to_file("data/top5_bus_routes.geojson", driver="GeoJSON")

In [ ]:
#DTL
buffer_overlap_400 = bus_routes_ls.copy()
buffer_overlap_400[['Overlap Length', 'Overlap Percentage']] = buffer_overlap_400['geometry'].apply(calculate_overlap)
service_overlap = buffer_overlap_400.groupby(['ServiceNo', 'Direction'])[['Overlap Length', 'Overlap Percentage']].sum().reset_index()
#print(service_overlap)
service_overlap_sorted = service_overlap.sort_values(by='Overlap Percentage', ascending=False)
service_overlap_sorted.head(10)

Filtering the bus routes for the shortlisted bus services

In [ ]:
TEL_interested_busroutes = TEL_top10_bus_routes[
    (TEL_top10_bus_routes["ServiceNo"].isin(['36B', '47', '196e', '901'])) &
    (TEL_top10_bus_routes["Direction"] == 1)
]

TEL_bus_routes_combined_filtered = bus_routes_combined[
    bus_routes_combined[['ServiceNo', 'Direction']].apply(tuple, axis=1).isin(TEL_interested_busroutes[['ServiceNo', 'Direction']].apply(tuple, axis=1))
]

# print(TEL_bus_routes_combined_filtered.head())

In [ ]:
#dtl
top5_overlap = service_overlap_sorted.head(5)['ServiceNo'].tolist()
top5_bus_routes = routes[routes['ServiceNo'].isin(top5_overlap)]

In [ ]:
# Function to check if a bus stop point is within the buffer
def is_within_buffer(bus_stop):
    return TE_buffer_400.contains(bus_stop)

# Create a new column 'Within_Buffer' for all bus routes
TEL_bus_routes_combined_filtered['Within_Buffer'] = bus_routes_combined['geometry'].apply(lambda x: TE_buffer_400.contains(x))

# Display the DataFrame with the new column
print(TEL_bus_routes_combined_filtered[['ServiceNo', 'Direction', 'geometry', 'Within_Buffer']].head())

# Count the number of bus stops within the buffer for each bus route
bus_stops_within_buffer_count = TEL_bus_routes_combined_filtered[TEL_bus_routes_combined_filtered['Within_Buffer']].groupby('ServiceNo').size().reset_index(name='Count_With_Buffer')

# Count the total number of bus stops for each bus route
total_bus_stops_count = TEL_bus_routes_combined_filtered.groupby('ServiceNo').size().reset_index(name='Total_Count')

# Merge the two DataFrames on 'ServiceNo'
bus_stops_summary = pd.merge(total_bus_stops_count, bus_stops_within_buffer_count, on='ServiceNo', how='left')

# Fill NaN values with 0 for routes that have no stops within the buffer
bus_stops_summary['Count_With_Buffer'] = bus_stops_summary['Count_With_Buffer'].fillna(0)

# Display the summary DataFrame
print(bus_stops_summary)

DTL map showing most overlapping bus routes

In [ ]:
sgmap = folium.Map(location=[1.3521, 103.8198], zoom_start=12)

coords = [(lat, lon) for lon,lat in route_line.coords]

folium.PolyLine(
    locations = coords,
    color='blue',
    weight=5,
    opacity=0.8,
    tooltip="Downtown Line"
).add_to(sgmap)

for _,row in DT_sorted.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['STN_NAME'],
        icon=folium.Icon(color='blue', icon='train', prefix='fa')
    ).add_to(sgmap)

In [ ]:
service_overlaplength_sorted = service_overlap.sort_values(by='Overlap Length', ascending=False)
service_overlaplength_sorted.head(10)

Identified bus service 36B for removal

In [ ]:

bus_stops_36B = bus_routes_combined[
        (bus_routes_combined['ServiceNo'] == "36B") &
        (bus_routes_combined['Direction'] == 1)
    ]

bus_stop_codes_36B = bus_stops_36B['BusStopCode'].unique().tolist()


In [ ]:
from geopy.distance import geodesic

def find_nearest_mrt(bus_stop_row, mrt_stations):
    bus_stop_location = (bus_stop_row['Latitude'], bus_stop_row['Longitude'])

    # Calculate distances to all MRT stations and find the nearest one
    mrt_stations['Distance'] = mrt_stations.apply(
        lambda row: geodesic(bus_stop_location, (row['Latitude'], row['Longitude'])).m,  # Ensure .m is used for meters
        axis=1
    )

    # Find the nearest MRT station by selecting the station with the minimum distance
    nearest_station = mrt_stations.loc[mrt_stations['Distance'].idxmin()]

    return nearest_station['STN_NO'], nearest_station['Distance']

# Apply the function to each bus stop and expand the results into two new columns
bus_stops_36B[['Nearest_MRT_Station', 'Distance_to_MRT']] = bus_stops_36B.apply(
    find_nearest_mrt, mrt_stations=mrt_gdf, axis=1, result_type='expand'
)

# Add a new column 'Near_MRT' based on the distance to MRT
bus_stops_36B['Near_MRT'] = bus_stops_36B['Distance_to_MRT'] <= 400

# Display or work with the updated DataFrame
print(bus_stops_36B.head())

# Count the number of bus stops that are near MRT stations
count_near_mrt = bus_stops_36B['Near_MRT'].sum()

# Display the count
print(f"Number of bus stops near MRT stations: {count_near_mrt}")


In [ ]:

bus_stops_36B = pd.merge(
    bus_stops_36B,
    mrt_stations[['STN_NO', 'STN_NAME']],
    left_on='Nearest_MRT_Station',
    right_on='STN_NO',
    how='left'
).drop(columns=['STN_NO'])

# bus_stops_36B.to_csv('data/bus_stops_36B.csv', index=False)


Function to find direct routes between 2 bus stops

In [ ]:

def find_routes(bus_routes_combined, start_bus_stop, end_bus_stop, exclude_routes=None):

    # Convert exclude_routes to a set if None is provided
    if exclude_routes is None:
        exclude_routes = set()

    # Find all routes serving the start and end bus stops
    routes_from_start = bus_routes_combined[
        bus_routes_combined['BusStopCode'] == start_bus_stop]['ServiceNo'].unique()

    routes_from_end = bus_routes_combined[
        bus_routes_combined['BusStopCode'] == end_bus_stop]['ServiceNo'].unique()

    # Exclude specified routes from the lists
    routes_from_start = set(routes_from_start) - exclude_routes
    routes_from_end = set(routes_from_end) - exclude_routes

    # Check for direct routes
    direct_routes = routes_from_start.intersection(routes_from_end)

    if direct_routes:
        print(f"Direct routes between {start_bus_stop} and {end_bus_stop}: {direct_routes}")
    else:
        print(f"No direct routes. Searching for alternative transfer routes...")

        # Find possible transfer points
        routes_from_start_df = bus_routes_combined[
            bus_routes_combined['ServiceNo'].isin(routes_from_start)]

        routes_from_end_df = bus_routes_combined[
            bus_routes_combined['ServiceNo'].isin(routes_from_end)]

        # Merge to find common transfer stops
        transfer_stops = pd.merge(
            routes_from_start_df, routes_from_end_df,
            on='BusStopCode', suffixes=('_start', '_end')
        )

        if not transfer_stops.empty:
            print("Transfer points and routes:")
            print(transfer_stops[['BusStopCode', 'ServiceNo_start', 'ServiceNo_end']].drop_duplicates())
        else:
            print("No valid transfer points found.")


API call to obtain token for OneMap routing API

In [ ]:
import requests
import json


# Set the URL and headers
url = "https://developers.onemap.sg/privateapi/auth/post/getToken"
headers = {
    "Content-Type": "application/json"
}

payload = {
    "email": "",
    "password": ""
}

# Send POST request
response = requests.post(url, headers=headers, data=json.dumps(payload))

# Check if request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    access_token = data['access_token']
    expiry_timestamp = data['expiry_timestamp']

    print(f"Access Token: {access_token}")
    print(f"Expiry: {expiry_timestamp}")
else:
    print(f"Failed to retrieve token. Status code: {response.status_code}")
    print(f"Response: {response.text}")


get onemap routes

In [ ]:
load_dotenv()

onemap_api_key = os.getenv('OneMap_api_key')

In [ ]:
import requests

# Define constant parameters
DATE = '10-21-2024'
TIME = '08:30:00'
MODE = 'TRANSIT'
NUM_ITINERARIES = '3'
MAX_WALK = '400'

API_TOKEN = onemap_api_key

# Function to get bus route data
def get_route(start_coords, end_coords):
    url = "https://www.onemap.gov.sg/api/public/routingsvc/route"
    headers = {
        "Authorization": API_TOKEN
    }
    params = {
        "start": f"{start_coords[0]},{start_coords[1]}",
        "end": f"{end_coords[0]},{end_coords[1]}",
        "routeType": "pt",
        "date": DATE,
        "time": TIME,
        "mode": MODE,
        "numItineraries": NUM_ITINERARIES,
        "maxWalkDistance": MAX_WALK,
    }
    try:
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            return response.json()  # Return the entire JSON response
        else:
            print(f"Error: Received status code {response.status_code}")
            print(f"Response: {response.text}")  # Log the response for debugging
    except requests.exceptions.RequestException as e:
        print(f"Error making request to OneMap API: {e}")
    return None

def format_duration(seconds):
    """Convert seconds to a formatted string of hours, minutes, and seconds."""
    return str(timedelta(seconds=seconds))

def print_itineraries(data):
    """
    Extracts and prints itinerary details from OneMap API response data.

    Parameters:
    - data: dict, the JSON response from the OneMap API, converted to a dictionary.
    """
    # Get the itineraries from the response
    itineraries = data.get('plan', {}).get('itineraries', [])

    if not itineraries:
        print("No itineraries found.")
        return

    for idx, itinerary in enumerate(itineraries):
        # Itinerary-level details
        total_duration = itinerary.get('duration', 0)
        transit_time = itinerary.get('transitTime', 0)
        walk_time = itinerary.get('walkTime', 0)
        fare = itinerary.get('fare', 'N/A')

        # Format duration and times
        formatted_duration = format_duration(total_duration)
        formatted_transit_time = format_duration(transit_time)
        formatted_walk_time = format_duration(walk_time)
        formatted_fare = f"${fare:.2f}" if isinstance(fare, (int, float)) else fare

        # Print summary of the itinerary
        print(f"\n--- Itinerary {idx + 1} ---")
        print(f"Total Duration: {formatted_duration}")
        print(f"Transit Time: {formatted_transit_time}")
        print(f"Walk Time: {formatted_walk_time}")
        print(f"Fare: {formatted_fare}")

        # Iterate over the legs in the itinerary
        for leg_idx, leg in enumerate(itinerary.get('legs', [])):
            mode = leg.get('mode', 'N/A')
            route = leg.get('route', 'N/A')
            from_stop = leg.get('from', {}).get('name', 'Unknown')
            to_stop = leg.get('to', {}).get('name', 'Unknown')
            distance = leg.get('distance', 0)
            start_time = leg.get('startTime', 'Unknown')
            end_time = leg.get('endTime', 'Unknown')
            leggeometry = leg.get('legGeometry', {}).get('points', [])
            num_stops = len(leg.get('intermediateStops', []))  # Count intermediate stops

            # Print details for each leg
            print(f"\nLeg {leg_idx + 1}:")
            print(f"  Mode: {mode}")
            print(f"  Route: {route if route else 'N/A'}")
            print(f"  From: {from_stop}")
            print(f"  To: {to_stop}")
            print(f"  Distance: {distance:.2f} meters")
            print(f"  Start Time: {start_time}")
            print(f"  End Time: {end_time}")
            if num_stops > 0:
                print(f"  Number of Intermediate Stops: {num_stops}")
            else:
                print(f"  No Intermediate Stops")
            if leggeometry:
                print(f"  Leg Geometry: {leggeometry} ")

    print("\n--- End of Itineraries ---")

def get_coordinates(bus_stop_code):
    # Look for the row in bus_stops_36B that matches the bus_stop_code
    stop = bus_stops[bus_stops['BusStopCode'] == bus_stop_code]

    # If a matching row is found, return the coordinates (latitude, longitude)
    if not stop.empty:
        lat = stop.iloc[0]['Latitude']
        lon = stop.iloc[0]['Longitude']
        return (lat, lon)
    else:
        return None  # Return None if the bus stop code is not found


In [ ]:
# Define start and end coordinates
start_coords =  get_coordinates(93019)  # Example starting point
end_coords = (1.292936722,103.8525856)  # Example ending point

data = get_route(start_coords, end_coords)
print_itineraries(data)


create passenger volume datasets

In [ ]:
passenger_volume_am = passenger_volume[
    (passenger_volume['TIME_PER_HOUR'].isin([8,9])) &
    (passenger_volume["DAY_TYPE"] == "WEEKDAY")]


passenger_volume_pm = passenger_volume[
    (passenger_volume['TIME_PER_HOUR'].isin([5,6])) &
    (passenger_volume["DAY_TYPE"] == "WEEKDAY")]

# Group by PT_CODE and aggregate both TOTAL_TAP_IN_VOLUME and TOTAL_TAP_OUT_VOLUME
tap_volume_am = passenger_volume_am.groupby('PT_CODE').agg({
    'TOTAL_TAP_IN_VOLUME': 'sum',
    'TOTAL_TAP_OUT_VOLUME': 'sum'
}).reset_index()

# Same for the PM period
tap_volume_pm = passenger_volume_pm.groupby('PT_CODE').agg({
    'TOTAL_TAP_IN_VOLUME': 'sum',
    'TOTAL_TAP_OUT_VOLUME': 'sum'
}).reset_index()

tap_volume_am.rename(columns={
    'TOTAL_TAP_IN_VOLUME': 'TOTAL_TAP_IN_VOLUME_am',
    'TOTAL_TAP_OUT_VOLUME': 'TOTAL_TAP_OUT_VOLUME_am'
}, inplace=True)

# Rename the columns in the PM dataset
tap_volume_pm.rename(columns={
    'TOTAL_TAP_IN_VOLUME': 'TOTAL_TAP_IN_VOLUME_pm',
    'TOTAL_TAP_OUT_VOLUME': 'TOTAL_TAP_OUT_VOLUME_pm'
}, inplace=True)

# Merge the passenger demand with bus stops
bus_stops_passenger_demand = bus_routes_combined.merge(
    tap_volume_am, left_on='BusStopCode', right_on='PT_CODE', how='left'
)
# Merge again with tap volume for PM period and add suffixes for PM columns
bus_stops_passenger_demand = bus_stops_passenger_demand.merge(
    tap_volume_pm, left_on='BusStopCode', right_on='PT_CODE', how='left', suffixes=('', '_pm'),
)

# Clean up duplicate PT_CODE columns (from both merges)
bus_stops_passenger_demand.drop(columns=['PT_CODE', 'PT_CODE_pm'], inplace=True)

bus_stops_gdf = gpd.GeoDataFrame(bus_stops_passenger_demand, geometry='geometry', crs="EPSG:4326")

tap in tap out map

In [ ]:
planning_areas = gpd.read_file("data/MasterPlan2019PlanningAreaBoundaryNoSea.geojson")

# Extract sub zone
planning_areas['sub_zone'] = planning_areas['Description'].str.extract(r'<th>SUBZONE_N</th>\s*<td>(.*?)</td>', expand=False).str.lower()

# Extract planning area
planning_areas['planning_area'] = planning_areas['Description'].str.extract(r'<th>PLN_AREA_N</th>\s*<td>(.*?)</td>', expand=False).str.lower()

# Extract region
planning_areas['region'] = planning_areas['Description'].str.extract(r'<th>REGION_N</th>\s*<td>(.*?)</td>', expand=False).str.lower()

planning_areas = planning_areas.to_crs(epsg=32648)
bus_stops_gdf = bus_stops_gdf.to_crs(epsg=32648)

bus_within_areas = gpd.sjoin(bus_stops_gdf, planning_areas, how="inner", predicate="within")

total_tap_in_volume_am = bus_within_areas.groupby('planning_area').agg(total_tap_in_volume_am=('TOTAL_TAP_IN_VOLUME_am', 'mean')).reset_index()
total_tap_out_volume_am = bus_within_areas.groupby('planning_area').agg(total_tap_out_volume_am=('TOTAL_TAP_OUT_VOLUME_am', 'mean')).reset_index()

tap_in_am = planning_areas.merge(total_tap_in_volume_am, on='planning_area', how='left')
tap_out_am = planning_areas.merge(total_tap_out_volume_am, on='planning_area', how='left')

# Fill NaN with 0 if necessary
tap_in_am['total_tap_in_volume_am'].fillna(0, inplace=True)
tap_out_am['total_tap_out_volume_am'].fillna(0, inplace=True)

# Convert planning areas GeoDataFrame to GeoJSON for Folium
tap_in_am_geojson = tap_in_am.to_crs(epsg=4326).to_json()
tap_out_am_geojson = tap_out_am.to_crs(epsg=4326).to_json()

# Initialize map centered around Singapore
tap_in_am_map = folium.Map(location=[1.3521, 103.8198], zoom_start=11)

# Add choropleth layer for total tap-in volume
Choropleth(
    geo_data=tap_in_am_geojson,
    data=tap_in_am,
    columns=['planning_area', 'total_tap_in_volume_am'],
    key_on='feature.properties.planning_area',
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Total Tap-In Volume AM',
).add_to(tap_in_am_map)

# Display the map
tap_in_am_map.save("tap_in_volume_map.html")

# Initialize map centered around Singapore
tap_out_am_map = folium.Map(location=[1.3521, 103.8198], zoom_start=11)

# Add choropleth layer for total tap-in volume
Choropleth(
    geo_data=tap_out_am_geojson,
    data=tap_out_am,
    columns=['Name', 'total_tap_out_volume_am'],
    key_on='feature.properties.Name',
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Total Tap-Out Volume AM',
).add_to(tap_out_am_map)

# Display the map
tap_out_am_map.save("tap_out_volume_map.html")
# tap_out_am_map

tap_in_am_map

accessibility map

In [ ]:

# Convert bus stops to GeoDataFrame
bus_stops['geometry'] = bus_stops.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
bus_stops_gdf = gpd.GeoDataFrame(bus_stops, geometry='geometry', crs="EPSG:4326")

# Convert MRT stations to GeoDataFrame
mrt_stations['geometry'] = mrt_stations.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
mrt_stations_gdf = gpd.GeoDataFrame(mrt_stations, geometry='geometry', crs="EPSG:4326")


# Ensure the planning areas are in a projected CRS for area calculations (e.g., UTM zone 48N for Singapore)
# You may need to replace 'epsg:32648' with the CRS suitable for your data location.
planning_areas = planning_areas.to_crs(epsg=32648)

# Calculate area in square kilometers
planning_areas['area_sq_km'] = planning_areas['geometry'].area / 1e6  # Convert from square meters to square km

# Reproject MRT stations and bus stops to match the planning areas CRS (EPSG:32648)
mrt_stations_gdf = mrt_stations_gdf.to_crs(epsg=32648)
bus_stops_gdf = bus_stops_gdf.to_crs(epsg=32648)

# Now proceed with the spatial joins
mrt_within_areas = gpd.sjoin(mrt_stations_gdf, planning_areas, how="inner", predicate="within")
mrt_counts = mrt_within_areas.groupby('Name').size().reset_index(name='mrt_count')

bus_within_areas = gpd.sjoin(bus_stops_gdf, planning_areas, how="inner", predicate="within")
bus_counts = bus_within_areas.groupby('Name').size().reset_index(name='bus_count')

# Merge counts with planning areas and continue with the rest of the steps
accessibility = planning_areas.merge(mrt_counts, on="Name", how="left")
accessibility = accessibility.merge(bus_counts, on="Name", how="left")

# Fill NaN with 0 and calculate densities
accessibility[['mrt_count', 'bus_count']] = accessibility[['mrt_count', 'bus_count']].fillna(0)
accessibility['mrt_density'] = accessibility['mrt_count'] / accessibility['area_sq_km']
accessibility['bus_density'] = accessibility['bus_count'] / accessibility['area_sq_km']
accessibility['normalized_accessibility_index'] = accessibility['mrt_density'] + accessibility['bus_density']


import folium
from folium import Choropleth

# Convert GeoDataFrame to GeoJSON
accessibility_geojson = accessibility.to_crs(epsg=4326).to_json()

# Initialize map centered around Singapore
m = folium.Map(location=[1.3521, 103.8198], zoom_start=11)

# Add choropleth layer for accessibility index
Choropleth(
    geo_data=accessibility_geojson,
    data=accessibility,
    columns=['Name', 'normalized_accessibility_index'],
    key_on='feature.properties.Name',
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Accessibility Index',
).add_to(m)

# Display the map
m.save("accessibility_map.html")

m

print(accessibility.head())

In [ ]:
def get_passenger_demand(bus_routes_combined, passenger_volume, year_month, service_no):
    # Filter the passenger_volume DataFrame
    passenger_volume_am = passenger_volume[
        (passenger_volume['TIME_PER_HOUR'].isin([8, 9])) &
        (passenger_volume["DAY_TYPE"] == "WEEKDAY") &
        (passenger_volume['YEAR_MONTH'] == year_month)
    ]

    # Group by PT_CODE and sum the relevant passenger volumes
    passenger_volume_am = passenger_volume_am.groupby('PT_CODE').agg({
        'TOTAL_TAP_IN_VOLUME': 'sum',
        'TOTAL_TAP_OUT_VOLUME': 'sum'
    }).reset_index()

    # Count the number of services for each bus stop
    service_counts = bus_routes_combined.groupby('BusStopCode')['ServiceNo'].nunique().reset_index()
    service_counts.columns = ['BusStopCode', 'ServiceCount']

    avg_tap_volume = pd.merge(service_counts, passenger_volume_am, left_on="BusStopCode", right_on='PT_CODE', how='left')

    # Clean up duplicate PT_CODE columns (from both merges)
    avg_tap_volume.drop(columns=['PT_CODE'], inplace=True)

    avg_tap_volume['avg_tap_in'] = avg_tap_volume['TOTAL_TAP_IN_VOLUME'] / avg_tap_volume['ServiceCount']
    avg_tap_volume['avg_tap_out'] = avg_tap_volume['TOTAL_TAP_OUT_VOLUME'] / avg_tap_volume['ServiceCount']

    # Fill NaN values with 0 for average calculations
    avg_tap_volume.fillna(0, inplace=True)

    avg_tap_volume['avg_tap_in'] = avg_tap_volume['avg_tap_in'].round().astype(int)
    avg_tap_volume['avg_tap_out'] = avg_tap_volume['avg_tap_out'].round().astype(int)

    bus_stops_passenger_demand = bus_routes_combined.merge(
        avg_tap_volume, on='BusStopCode', how='left'
    )

    # Filter for the specified service number and direction (1)
    passenger_demand = bus_stops_passenger_demand[
        (bus_stops_passenger_demand['ServiceNo'] == service_no) &
        (bus_stops_passenger_demand['Direction'] == 1)
    ]

    # Ensure the average tap-in volumes are rounded to whole numbers
    passenger_demand['avg_tap_in'] = passenger_demand['avg_tap_in'].round()
    passenger_demand['avg_tap_out'] = passenger_demand['avg_tap_out'].round()

    # Set the last observation of avg_tap_in to 0
    passenger_demand['avg_tap_in'].iloc[-1] = 0
    passenger_demand['avg_tap_out'].iloc[0] = 0

    # Sort the DataFrame by StopSequence for ordered plotting
    passenger_demand = passenger_demand.sort_values(by='StopSequence')

    # Convert BusStopCode to a categorical variable ordered by StopSequence
    passenger_demand['BusStopCode'] = pd.Categorical(passenger_demand['BusStopCode'],
                                                       categories=passenger_demand['BusStopCode'].unique(),
                                                       ordered=True)

    return passenger_demand



In [ ]:
import matplotlib.pyplot as plt

# Get passenger demand data for the three months
service_no = "36B"
passenger_demand_36B_jul = get_passenger_demand(bus_routes_combined, passenger_volume, "2024-07", service_no)
passenger_demand_36B_aug = get_passenger_demand(bus_routes_combined, passenger_volume, "2024-08", service_no)
passenger_demand_36B_sept = get_passenger_demand(bus_routes_combined, passenger_volume, "2024-09", service_no)

# Create side-by-side subplots
fig, axs = plt.subplots(1, 2, figsize=(16, 6))

# Plot average tap-in volumes
axs[0].plot(range(len(passenger_demand_36B_jul)), passenger_demand_36B_jul['avg_tap_in'], marker='o', color='blue', linestyle='-', label='July')
axs[0].plot(range(len(passenger_demand_36B_aug)), passenger_demand_36B_aug['avg_tap_in'], marker='o', color='orange', linestyle='-', label='August')
axs[0].plot(range(len(passenger_demand_36B_sept)), passenger_demand_36B_sept['avg_tap_in'], marker='o', color='green', linestyle='-', label='September')

# Add labels and title for tap-in plot
axs[0].set_xlabel('Bus Stop Code')
axs[0].set_ylabel('Average Tap-In Volume')
axs[0].set_title('Average Tap-In Volume for Bus Service 36B by Stop Sequence')
axs[0].set_xticks(range(len(passenger_demand_36B_jul)))
axs[0].set_xticklabels(passenger_demand_36B_jul['BusStopCode'],  rotation=90, ha='right')
axs[0].grid(axis='y')
axs[0].legend()

# Plot average tap-out volumes
axs[1].plot(range(len(passenger_demand_36B_jul)), passenger_demand_36B_jul['avg_tap_out'], marker='o', color='blue', linestyle='-', label='July')
axs[1].plot(range(len(passenger_demand_36B_aug)), passenger_demand_36B_aug['avg_tap_out'], marker='o', color='orange', linestyle='-', label='August')
axs[1].plot(range(len(passenger_demand_36B_sept)), passenger_demand_36B_sept['avg_tap_out'], marker='o', color='green', linestyle='-', label='September')

# Add labels and title for tap-out plot
axs[1].set_xlabel('Bus Stop Code')
axs[1].set_ylabel('Average Tap-Out Volume')
axs[1].set_title('Average Tap-Out Volume for Bus Service 36B by Stop Sequence')
axs[1].set_xticks(range(len(passenger_demand_36B_jul)))
axs[1].set_xticklabels(passenger_demand_36B_jul['BusStopCode'],  rotation=90, ha='right')
axs[1].grid(axis='y')
axs[1].legend()

# Show the plots
plt.tight_layout()  # Adjust layout to prevent clipping of tick-labels
plt.show()


In [ ]:

passenger_volume_OD = passenger_volume_OD[
        (passenger_volume_OD['TIME_PER_HOUR'].isin([8, 9])) &
        (passenger_volume_OD["DAY_TYPE"] == "WEEKDAY") &
        (passenger_volume_OD['YEAR_MONTH'] == "2024-09")
    ]

    # Group by PT_CODE and sum the relevant passenger volumes
passenger_volume_OD = passenger_volume_OD.groupby(['ORIGIN_PT_CODE', 'DESTINATION_PT_CODE']).agg({
    'TOTAL_TRIPS': 'sum',
}).reset_index()

    # Count the number of services for each bus stop
service_counts = bus_routes_combined.groupby('BusStopCode')['ServiceNo'].nunique().reset_index()
service_counts.columns = ['BusStopCode', 'ServiceCount']

avg_tap_volume = pd.merge(service_counts, passenger_volume_am, left_on="BusStopCode", right_on='PT_CODE', how='left')

od_pairs = []

bus_stops_36B = bus_routes_combined[
        (bus_routes_combined['ServiceNo'] == service_no) &
        (bus_routes_combined['Direction'] == 1)
    ]

# Generate pairs of bus stops
for (i, row1), (j, row2) in combinations(bus_stops_36B.iterrows(), 2):
    if row1['StopSequence'] < row2['StopSequence']:  # Ensure origin has a smaller StopSequence
        od_pairs.append((row1['BusStopCode'], row2['BusStopCode']))

# Convert the list of pairs to a DataFrame
od_pairs_df = pd.DataFrame(od_pairs, columns=['Origin', 'Destination'])

# Display the resulting DataFrame of origin-destination pairs
# print(od_pairs_df)

import pandas as pd
from itertools import combinations

# Assuming bus_routes_combined is your DataFrame and service_no is defined
service_no = "36B"

# Filter for bus stops for service '36B' in the specified direction (1)
bus_stops_36B = bus_routes_combined[
    (bus_routes_combined['ServiceNo'] == service_no) &
    (bus_routes_combined['Direction'] == 1)
]

# Generate pairs of bus stops
od_pairs = []
for (i, row1), (j, row2) in combinations(bus_stops_36B.iterrows(), 2):
    if row1['StopSequence'] < row2['StopSequence']:  # Ensure origin has a smaller StopSequence
        od_pairs.append((row1['BusStopCode'], row2['BusStopCode']))

# Convert the list of pairs to a DataFrame
od_pairs_df = pd.DataFrame(od_pairs, columns=['Origin', 'Destination'])

# Initialize a list to store results
results = []

# For each OD pair, find the count of unique services
for index, row in od_pairs_df.iterrows():
    origin = row['Origin']
    destination = row['Destination']

    # Filter services that go through both origin and destination
    services_count = bus_routes_combined[
        ((bus_routes_combined['BusStopCode'] == origin) | (bus_routes_combined['BusStopCode'] == destination))
    ]['ServiceNo'].nunique()  # Count unique services

    # Append the result
    results.append((origin, destination, services_count))

# Convert the results to a DataFrame
results_df = pd.DataFrame(results, columns=['Origin', 'Destination', 'ServiceCount'])

# Display the results
print(results_df)


High demand origins and destinations

In [ ]:
import pandas as pd

# Load the OD dataset
od_data = pd.read_csv('data/passenger_volume_OD.csv')
# Total trips by origin bus stop
origin_demand = od_data.groupby('ORIGIN_PT_CODE')['TOTAL_TRIPS'].sum().reset_index()
origin_demand.rename(columns={'TOTAL_TRIPS': 'TOTAL_TRIPS_FROM_ORIGIN'}, inplace=True)

# Total trips by destination bus stop
destination_demand = od_data.groupby('DESTINATION_PT_CODE')['TOTAL_TRIPS'].sum().reset_index()
destination_demand.rename(columns={'TOTAL_TRIPS': 'TOTAL_TRIPS_TO_DESTINATION'}, inplace=True)

# Calculate threshold (for example, 75th percentile)
origin_threshold = origin_demand['TOTAL_TRIPS_FROM_ORIGIN'].quantile(0.90)
destination_threshold = destination_demand['TOTAL_TRIPS_TO_DESTINATION'].quantile(0.90)

# Identify high-demand origins and destinations
high_demand_origins = origin_demand[origin_demand['TOTAL_TRIPS_FROM_ORIGIN'] >= origin_threshold]
high_demand_destinations = destination_demand[destination_demand['TOTAL_TRIPS_TO_DESTINATION'] >= destination_threshold]

# Merge high-demand origins with coordinates
high_demand_origins = high_demand_origins.merge(bus_stops, left_on='ORIGIN_PT_CODE', right_on='BusStopCode', how='left')

# Merge high-demand destinations with coordinates
high_demand_destinations = high_demand_destinations.merge(bus_stops, left_on='DESTINATION_PT_CODE', right_on='BusStopCode', how='left')

# Convert origins and destinations into GeoDataFrames
# Assuming you have their coordinates (e.g., latitude and longitude) in the DataFrame
origins_gdf = gpd.GeoDataFrame(high_demand_origins, geometry=gpd.points_from_xy(high_demand_origins.Longitude, high_demand_origins.Latitude), crs='EPSG:4326')
destinations_gdf = gpd.GeoDataFrame(high_demand_destinations, geometry=gpd.points_from_xy(high_demand_destinations.Longitude, high_demand_destinations.Latitude), crs='EPSG:4326')

# Set the same CRS for planning areas, origins, and destinations if they differ
planning_areas = planning_areas.to_crs(epsg=4326)

# Plotting
fig, ax = plt.subplots(figsize=(10, 10))
planning_areas.plot(ax=ax, color='lightgrey', edgecolor='black')
origins_gdf.plot(ax=ax, color='blue', marker='o', markersize=50, label='High Demand Origins')
destinations_gdf.plot(ax=ax, color='red', marker='x', markersize=50, label='High Demand Destinations')

plt.title('High Demand Origins and Destinations in Singapore')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend()
plt.show()

heatmap

In [ ]:


# Combine origins and destinations into one DataFrame
heat_data = pd.concat([
    high_demand_origins[['Latitude', 'Longitude']],
    high_demand_destinations[['Latitude', 'Longitude']]
], ignore_index=True)

# Create a base map
singapore_map = folium.Map(location=[1.3521, 103.8198], zoom_start=12)  # Centered on Singapore

# Prepare data for heatmap
heat_data = [[row['Latitude'], row['Longitude']] for index, row in heat_data.iterrows()]

# Create a heatmap layer
HeatMap(heat_data, radius=15, blur=10).add_to(singapore_map)

# Save the map to an HTML file or display it in a Jupyter notebook
singapore_map.save("singapore_heatmap.html")
singapore_map
